# CloudML with DataLAB Part. 6

### Hyperparameter Tuning

Hello. I'm back!

This session, we well learn about Hyperparameter Tuning on CloudML

If you designed a small model, It's fine. You can tune its hyperparameter simpley

But you don't need to use the CloudML in that case. It's enough to run the model in local.

Large and huge model requires a lot of hyperparameters and it's really hard to tune

You can do it autometically if you use Hyperparameter Tuning function in CloudML

Stop wasting your time! It's your time for sleep!

Here's simple code to do it

In [1]:
%%writefile ./model/hptuning.py
import tensorflow as tf
from tensorflow.contrib.learn import Experiment
from tensorflow.contrib.learn.python.learn import learn_runner
from tensorflow.contrib.learn.python.learn import Estimator
from tensorflow.contrib.learn import ModeKeys

import os
import json
import argparse

tf.logging.set_verbosity(tf.logging.INFO)

# all the function would be retured as function type
# so I also define the train_input function inside of train_input_fn function 
# that returns the trian_input as function type
def train_input_fn(args):
    def train_input():
        # genertate the train data (10000, 10) and label data (10000, 1)
        data = []
        label = []
        for i in range(100):
            if i % 2 == 0:
                # label 1 at increasing data 
                data.append([i for i in range(10)])
                label.append([1])
            else:
                # label 0 at decreasing data
                data.append([9-i for i in range(10)])
                label.append([0])

        return tf.constant(data, tf.float32), tf.constant(label, tf.float32)
    return train_input

# your model whatever you want
# you can see the input arguments of function pair
# I set the input argument of args which is got from shell input to model_fn input
# so I feel free to use it in the function
# and 3 arguments are in the model
# It's to be called with 3 arguments by experiment object 
def model_fn(args):
    def model(data, label, mode):
        metric = {}
        steps = 0
        # Build the model here.
        # hidden1 and hidden2 from args is to be a parameter which will be tuned
        layer1 = tf.layers.dense(data, 10, name="layer1")
        layer2 = tf.layers.dense(layer1, args.hidden1, name="layer2")
        layer3 = tf.layers.dense(layer2, args.hidden2, name="layer3")
        logits = tf.layers.dense(layer3, 1, name="logits")

        # build the graph following these case by case
        if mode == ModeKeys.INFER:
            metric['prediction'] = tf.argmax(tf.nn.sigmoid(logits), 1)
            loss = None
        else:
            loss = tf.losses.sigmoid_cross_entropy(label, logits)
            metric['loss'] = loss

        if mode == ModeKeys.TRAIN:
            global_step = tf.contrib.framework.get_global_step()
            train = tf.train.GradientDescentOptimizer(0.00001).minimize(loss, global_step)
        else:
            train = None

        if mode == ModeKeys.EVAL:
            prediction = tf.nn.sigmoid(logits)
            accuracy = tf.contrib.metrics.accuracy(tf.cast(tf.round(prediction), tf.int32), tf.cast(label, tf.int32))
            metric['accuracy'] = accuracy
            # This line is for the hyperparameter tuning
            # the cloudml always checks the summaries
            # so you just add summary like this then cloudML would see it 
            tf.summary.scalar('accuracy', accuracy)

        return metric, loss, train
    return model

# The experiment function would designed simpley like below
def experiment_fn(args):
    print("exp:", args)
    def experiment(output_dir):
        env = json.loads(os.environ.get('TF_CONFIG', '{}'))
        taskInfo = env.get('task')
        if taskInfo:
            trial = taskInfo.get('trial', '')
            if trial:
                output_dir = os.path.join(output_dir, trial)
            
        return tf.contrib.learn.Experiment(
            estimator=Estimator(
                model_fn=model_fn(args),
                model_dir=output_dir
            ),
            train_input_fn=train_input_fn(args),
            eval_input_fn=train_input_fn(args),
            train_steps=10,
            eval_steps=5
        )
    return experiment

# Add the argument "hidden1" and "hidden2" 
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    # Training arguments
    parser.add_argument(
        '--hidden1',
        type=int,
        required=True
    )
    parser.add_argument(
        '--hidden2',
        type=int,
        required=True
    )
    parser.add_argument(
        '--output-dir',
        required=True
    )

    args = parser.parse_args()
    output_dir = args.output_dir

    learn_runner.run(experiment_fn(args), output_dir)

Writing ./model/hptuning.py


Check for errors and write this file below

This is key-point of this session

hyperparameterMetricTag must be same as summary name you added upon the code above

And parameterName is the arguments you set. Of course both names are same also

Please care for the indentation

In [3]:
%%writefile ./model/hpconfig.yaml
trainingInput:
  hyperparameters:
    goal: MAXIMIZE
    hyperparameterMetricTag: accuracy
    maxTrials: 4
    maxParallelTrials: 2
    params:
      - parameterName: hidden1
        type: INTEGER
        minValue: 1
        maxValue: 10
        scaleType: UNIT_LINEAR_SCALE
      - parameterName: hidden2
        type: INTEGER
        minValue: 1
        maxValue: 10
        scaleType: UNIT_LINEAR_SCALE

Writing ./model/hpconfig.yaml


if done until right here, run!

In [14]:
%bash
PROJECT_ID=tensorflowprojects
BUCKET="gs://${PROJECT_ID}-mlengine"
JOB_NAME=simple_model_$(date +%Y%m%d_%H%M%S)
PACKAGE_PATH=$(pwd)/model
MODULE_NAME=model.hptuning
STAGING_BUCKET=${BUCKET}
JOB_DIR=${BUCKET}/${JOB_NAME}
OUTPUT=${BUCKET}/${JOB_NAME}
REGION=europe-west1
SCALE_TIER=BASIC_GPU
CONFIG=${PACKAGE_PATH}/hpconfig.yaml

# Submit job with these settings
gcloud ml-engine jobs submit training $JOB_NAME \
--package-path=$PACKAGE_PATH \
--module-name=$MODULE_NAME \
--staging-bucket=$STAGING_BUCKET \
--scale-tier=$SCALE_TIER \
--region=$REGION \
--config=$CONFIG \
-- \
--output-dir=$OUTPUT

jobId: simple_model_20170614_225919
state: QUEUED


Job [simple_model_20170614_225919] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe simple_model_20170614_225919

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs simple_model_20170614_225919


You can see whether it runs as hyperparameter tuning job through "describe" command

In [12]:
!gcloud ml-engine jobs describe simple_model_20170614_225410

createTime: '2017-06-14T22:54:12Z'
jobId: simple_model_20170614_225410
startTime: '2017-06-14T22:54:22Z'
state: RUNNING
trainingInput:
  args:
  - --output-dir=gs://tensorflowprojects-mlengine/simple_model_20170614_225410
  hyperparameters:
    goal: MAXIMIZE
    hyperparameterMetricTag: accuracy
    maxParallelTrials: 2
    maxTrials: 4
    params:
    - maxValue: 10.0
      minValue: 1.0
      parameterName: hidden1
      scaleType: UNIT_LINEAR_SCALE
      type: INTEGER
    - maxValue: 10.0
      minValue: 1.0
      parameterName: hidden2
      scaleType: UNIT_LINEAR_SCALE
      type: INTEGER
  jobDir: gs://tensorflowprojects-mlengine/simple_model_20170614_225410
  packageUris:
  - gs://tensorflowprojects-mlengine/simple_model_20170614_225410/3e80309342f8620a3597d16bb82c0c3ee9f3773134792e64afc4e8f22c1b77aa/model-0.0.0.tar.gz
  pythonModule: model.hptuning
  region: europe-west1
  scaleTier: BASIC_GPU
trainingOutput:
  isHyperparameterTuningJob: true

View job in the Cloud Console at:

Done! Easy isn't it?

I hope you to be help! thanks for reading a lot!

now, Enjoy!